# Weights and Biases

In [7]:
# modified from https://github.com/dmlc/xgboost/blob/master/demo/multiclass_classification/train.py

import wandb
import numpy as np
import xgboost as xgb

wandb.init()

# label need to be 0 to num_class -1
data = np.loadtxt('./dermatology.data', delimiter=',',
        converters={33: lambda x:int(x == '?'), 34: lambda x:int(x) - 1})
sz = data.shape

train = data[:int(sz[0] * 0.7), :]
test = data[int(sz[0] * 0.7):, :]

train_X = train[:, :33]
train_Y = train[:, 34]

test_X = test[:, :33]
test_Y = test[:, 34]

xg_train = xgb.DMatrix(train_X, label=train_Y)
xg_test = xgb.DMatrix(test_X, label=test_Y)
# setup parameters for xgboost
param = {}
# use softmax multi-class classification
param['objective'] = 'multi:softmax'
# scale weight of positive examples
param['eta'] = 0.1
param['max_depth'] = 6
param['silent'] = 1
param['nthread'] = 4
param['num_class'] = 6
wandb.config.update(param)

watchlist = [(xg_train, 'train'), (xg_test, 'test')]
num_round = 5
bst = xgb.train(param, xg_train, num_round, watchlist, callbacks=[wandb.xgboost.wandb_callback()])
# get prediction
pred = bst.predict(xg_test)
error_rate = np.sum(pred != test_Y) / test_Y.shape[0]
print('Test error using softmax = {}'.format(error_rate))
wandb.summary['Error Rate'] = error_rate

FileNotFoundError: ./dermatology.data not found.

In [6]:
import wandb
import xgboost as xgb
from wandb.xgboost import wandb_callback
import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn import svm, datasets
from sklearn.metrics import roc_curve, roc_auc_score, auc, classification_report, confusion_matrix, plot_roc_curve
from sklearn.metrics import PrecisionRecallDisplay, average_precision_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from xgboost import XGBClassifier
from collections import Counter
import warnings
import omdml 

warnings.filterwarnings('ignore')

# Import some data to play with
X, y = datasets.load_breast_cancer(return_X_y=True, as_frame=True)
df = X.copy()
df['target'] = y
df.sample(frac=1, replace=True, random_state=42)
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.3, random_state=42)

params = {
    'learning_rate': 0.1, 
    'n_estimators': 20, 
    'max_depth': 3, 
    'subsample': 0.7
}

xgb = XGBClassifier(**params, verbosity=0, random_state=42)
wandb.init(project="my-test-project")
bst = xgb.train(param, X_train, num_round, watchlist, callbacks=[wandb_callback()])
pred = bst.predict(xg_test)

AttributeError: 'XGBClassifier' object has no attribute 'train'